#**1.Unir secuencias**

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage import io
import glob
import nibabel as nib
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH_TRAIN = "/content/drive/MyDrive/Electivos/IAA/Proyecto/code/dataset/BraTS-2020/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
PATH_VALI = "/content/drive/MyDrive/Electivos/IAA/Proyecto/code/dataset/BraTS-2020/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"

## **A)Train**

In [19]:
os.chdir(PATH_TRAIN)
list_train = os.listdir(PATH_TRAIN)

In [ ]:
for i in range(len(list_train)):
  list_sec = os.listdir("./" + list_train[i])

  image_path_flair = list_sec[0]
  image_path_t1 = list_sec[2]
  image_path_seg = list_sec[1]
  image_path_t1ce = list_sec[3]
  image_path_t2 = list_sec[4]

  image_obj_flair = nib.load(PATH_TRAIN + "/"  + str(list_train[i]) + "/" + str(image_path_flair))
  image_obj_t1ce = nib.load(PATH_TRAIN + "/"  + str(list_train[i]) + "/" + str(image_path_t1ce))
  image_obj_t1 = nib.load(PATH_TRAIN + "/"  + str(list_train[i]) + "/" + str(image_path_t1))
  image_obj_t2 = nib.load(PATH_TRAIN + "/"  + str(list_train[i]) + "/" + str(image_path_t2))
  image_obj_seg = nib.load(PATH_TRAIN + "/"  + str(list_train[i]) + "/" + str(image_path_seg))

  image_data_flair = image_obj_flair.get_fdata()
  image_data_t1ce = image_obj_t1ce.get_fdata()
  image_data_t1 = image_obj_t1.get_fdata()
  image_data_t2 = image_obj_t2.get_fdata()
  image_path_seg = image_obj_seg.get_fdata()

  # Renombramos los valores de 4 con 3
  image_path_seg[image_path_seg == 4.] = 3.

  image_path_seg =nib.Nifti1Image(image_path_seg, np.eye(4))
  # Juntamos las 4 secuencias separadas en una solo arreglo
  total_BraTS20 = np.stack((image_data_flair,
                  image_data_t1ce,
                  image_data_t1,
                  image_data_t2),
                  axis = 3)
  total_BraTS20 = nib.Nifti1Image(total_BraTS20, np.eye(4))

  nib.save(total_BraTS20, 
           os.path.join(PATH + "/"  + str(list_train[i]), 'BraTS20_Training_'+ str(i+1).zfill(3) + '_total.nii')
           )
  
  nib.save(image_path_seg,
           os.path.join(PATH + "/"  + str(list_train[i]), 'BraTS20_Training_'+ str(i+1).zfill(3) + '_segf.nii')
           )

In [ ]:
!ls

In [23]:
# Comprobamos que se hayan guardado de manera correcta
image_path_flair = './BraTS20_Training_001/BraTS20_Training_001_flair.nii'
image_path_t1ce = './BraTS20_Training_001/BraTS20_Training_001_t1ce.nii'
image_path_t1 = './BraTS20_Training_001/BraTS20_Training_001_t1.nii'
image_path_t2 = './BraTS20_Training_001/BraTS20_Training_001_t2.nii'

image_obj_flair = nib.load(image_path_flair)
image_obj_t1ce = nib.load(image_path_t1ce)
image_obj_t1 = nib.load(image_path_t1)
image_obj_t2 = nib.load(image_path_t2)

In [24]:
image_data_flair = image_obj_flair.get_fdata()
image_data_t1ce = image_obj_t1ce.get_fdata()
image_data_t1 = image_obj_t1.get_fdata()
image_data_t2 = image_obj_t2.get_fdata()

type(image_data_flair)

numpy.memmap

In [25]:
image_path_total = './BraTS20_Training_001/BraTS20_Training_001_total.nii'
image_obj_total = nib.load(image_path_total)
image_data_total = image_obj_total.get_fdata()

In [26]:
total_BraTS20_Training_001 = np.stack((image_data_flair,
                                        image_data_t1ce,
                                        image_data_t1,
                                        image_data_t2),
                                        axis = 3)

In [28]:
total_BraTS20_Training_001.shape

(240, 240, 155, 4)

In [27]:
image_data_total.shape

(240, 240, 155, 4)

In [29]:
(total_BraTS20_Training_001 == image_data_total).all()

True

In [30]:
# Segmentacion final
image_path_segf = './BraTS20_Training_001/BraTS20_Training_001_segf.nii'
image_obj_segf = nib.load(image_path_segf)
image_data_segf = image_obj_segf.get_fdata()

In [31]:
image_data_segf.shape

(240, 240, 155)

In [38]:
# Segmentacion inicial
image_path_seg = './BraTS20_Training_001/BraTS20_Training_001_seg.nii'
image_obj_seg = nib.load(image_path_seg)
image_data_seg = image_obj_seg.get_fdata()

In [39]:
image_data_seg.shape

(240, 240, 155)

In [40]:
np.unique(image_data_seg)

array([0., 1., 2., 4.])

In [41]:
# Cambiamos los pixeles con valor de 4 con 3
image_data_seg[image_data_seg == 4.] = 3.

In [42]:
np.unique(image_data_seg)

array([0., 1., 2., 3.])

In [43]:
# Comprobamos si son iguales
(image_data_seg == image_data_segf).all()
# True: se guardo bien

True

## **B)Validation**

In [10]:
os.chdir(PATH_VALI)
list_vali = os.listdir(PATH_VALI)

In [ ]:
for i in range(len(list_vali)):
  list_sec = os.listdir("./" + list_vali[i])

  image_path_flair = list_sec[0]
  image_path_t1 = list_sec[1]
  image_path_t1ce = list_sec[2]
  image_path_t2 = list_sec[3]

  image_obj_flair = nib.load(PATH_VALI + "/"  + str(list_vali[i]) + "/" + str(image_path_flair))
  image_obj_t1ce = nib.load(PATH_VALI + "/"  + str(list_vali[i]) + "/" + str(image_path_t1ce))
  image_obj_t1 = nib.load(PATH_VALI + "/"  + str(list_vali[i]) + "/" + str(image_path_t1))
  image_obj_t2 = nib.load(PATH_VALI + "/"  + str(list_vali[i]) + "/" + str(image_path_t2))

  image_data_flair = image_obj_flair.get_fdata()
  image_data_t1ce = image_obj_t1ce.get_fdata()
  image_data_t1 = image_obj_t1.get_fdata()
  image_data_t2 = image_obj_t2.get_fdata()

  # Juntamos las 4 secuencias separadas en una solo arreglo
  total_BraTS20 = np.stack((image_data_flair,
                  image_data_t1ce,
                  image_data_t1,
                  image_data_t2),
                  axis = 3)
  total_BraTS20 = nib.Nifti1Image(total_BraTS20, np.eye(4))

  nib.save(total_BraTS20, 
           os.path.join(PATH_VALI + "/"  + str(list_vali[i]), 'BraTS20_Validation_'+ str(i+1).zfill(3) + '_total.nii')
           )
print("Fin!")

In [12]:
# Comprobamos que se hayan guardado de manera correcta
image_path_flair = './BraTS20_Validation_001/BraTS20_Validation_001_flair.nii'
image_path_t1ce = './BraTS20_Validation_001/BraTS20_Validation_001_t1ce.nii'
image_path_t1 = './BraTS20_Validation_001/BraTS20_Validation_001_t1.nii'
image_path_t2 = './BraTS20_Validation_001/BraTS20_Validation_001_t2.nii'

image_obj_flair = nib.load(image_path_flair)
image_obj_t1ce = nib.load(image_path_t1ce)
image_obj_t1 = nib.load(image_path_t1)
image_obj_t2 = nib.load(image_path_t2)

In [13]:
image_data_flair = image_obj_flair.get_fdata()
image_data_t1ce = image_obj_t1ce.get_fdata()
image_data_t1 = image_obj_t1.get_fdata()
image_data_t2 = image_obj_t2.get_fdata()

type(image_data_flair)

numpy.memmap

In [16]:
total_BraTS20_Validation_001 = np.stack((image_data_flair,
                                        image_data_t1ce,
                                        image_data_t1,
                                        image_data_t2),
                                        axis = 3)

In [17]:
total_BraTS20_Validation_001.shape

(240, 240, 155, 4)

In [14]:
image_path_total = './BraTS20_Validation_001/BraTS20_Validation_001_total.nii'
image_obj_total = nib.load(image_path_total)
image_data_total = image_obj_total.get_fdata()
image_data_total.shape

(240, 240, 155, 4)

In [18]:
(image_data_total == total_BraTS20_Validation_001).all()
# True : Se guardo bien

True